1: DimDate Table

In [ ]:
%pip install --upgrade sqlalchemy
%pip install --upgrade pyodbc

In [ ]:
import pyodbc
print(pyodbc.drivers())

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Maak een range van datums
date_list = pd.date_range(start="01-01-2010", end="31-12-2025", freq='D')

df_dimdate = pd.DataFrame({
    'DateKey': date_list.strftime('%Y%m%d').astype(int),  # YYYYMMDD als key
    'FullDate': date_list.date,  # Volledige datum
    'DayName': date_list.strftime('%A'),  # Dagnaam in Engels
    'MonthNameDutch': date_list.strftime('%B'),  # Maandnaam (kan vertaald worden)
    'MonthNameEN': date_list.strftime('%B'),  # Maandnaam in Engels
    'DayNameDutch': date_list.strftime('%A'),  # Dagnaam in Nederlands
    'DayNameEN': date_list.strftime('%A'),  # Dagnaam in Engels
    'QuarterName': 'Q' + date_list.quarter.astype(str),  # Kwartaal als 'Q1', 'Q2', ...
    'QuarterNumber': date_list.quarter  # Kwartaalnummer (1-4)
})

# Schrijf naar SQL Server

# Gegevens voor de verbinding
server = "xxx"  # Servernaam of IP-adres van je SQL Server
database = "DEP1_DWH"  # Naam van je database

# Maak de verbindingsstring met Windows Authenticatie (Integrated Security)
engine = create_engine("mssql+pyodbc://@{}/{}?driver=ODBC+Driver+17+for+SQL+Server".format(server, database))
df_dimdate.to_sql('DimDate', con=engine, if_exists='append', index=False)


19

In [2]:
df_dimdate.head()

,DateKey,FullDate,DayName,MonthNameDutch,MonthNameEN,DayNameDutch,DayNameEN,QuarterName,QuarterNumber
0,20100101,2010-01-01,Friday,January,January,Friday,Friday,Q1,1
1,20100102,2010-01-02,Saturday,January,January,Saturday,Saturday,Q1,1
2,20100103,2010-01-03,Sunday,January,January,Sunday,Sunday,Q1,1
3,20100104,2010-01-04,Monday,January,January,Monday,Monday,Q1,1
4,20100105,2010-01-05,Tuesday,January,January,Tuesday,Tuesday,Q1,1


2: DimTime Table

In [ ]:
# Genereer tijdstippen van 00:00 tot 23:59 per minuut
time_range = pd.date_range("00:00", "23:59", freq="T").time
dim_time_df = pd.DataFrame(time_range, columns=['Time'])

dim_time_df['TimeKey'] = dim_time_df['Time'].apply(lambda x: int(x.strftime('%H%M')))
dim_time_df['Hour'] = dim_time_df['Time'].apply(lambda x: x.hour)
dim_time_df['Minute'] = dim_time_df['Time'].apply(lambda x: x.minute)

# Schrijf naar SQL Server
dim_time_df.to_sql('DimTime', con=engine, if_exists='replace', index=False)


3: DimWeatherStation Table

In [ ]:
# Lees de CSV voor weerstations
weather_station_df = pd.read_csv('weather_stations.csv')

# Verwerk de kolommen
weather_station_df = weather_station_df[['StationName', 'Latitude', 'Longitude', 'Altitude']]

# Schrijf naar SQL Server
weather_station_df.to_sql('DimWeatherStation', con=engine, if_exists='replace', index=False)


4: FactWeather Table

In [ ]:
# Lees de CSV voor weerdata
weather_data_df = pd.read_csv('weather_data.csv')

# Voeg de foreign keys toe
weather_data_df['DateKey'] = pd.to_datetime(weather_data_df['Date']).dt.strftime('%Y%m%d')
weather_data_df['TimeKey'] = pd.to_datetime(weather_data_df['Time']).dt.strftime('%H%M')

# Voeg de WeatherStationKey toe (bijvoorbeeld door te matchen met de StationName)
weather_data_df = weather_data_df.merge(weather_station_df[['StationName', 'WeatherStationKey']], on='StationName', how='left')

# Schrijf naar SQL Server
weather_data_df.to_sql('FactWeather', con=engine, if_exists='replace', index=False)


5: FactBelpex Table

In [ ]:
# Lees de CSV voor BELPEX
belpex_df = pd.read_csv('belpex_data.csv')

# Voeg de foreign keys toe
belpex_df['DateKey'] = pd.to_datetime(belpex_df['Date']).dt.strftime('%Y%m%d')
belpex_df['TimeKey'] = pd.to_datetime(belpex_df['Time']).dt.strftime('%H%M')

# Schrijf naar SQL Server
belpex_df.to_sql('FactBelpex', con=engine, if_exists='replace', index=False)


6: Verbruikersdata

- Verwerk de verbruikersdata naar het juiste formaat.
- Gebruik SQL Server's bulk-insert om grote hoeveelheden data efficiënt te verwerken.

7: Overige Tabellen

- Voor de overige tabellen, volg dezelfde logica:
    - Lees de CSV’s.
    - Voeg de benodigde foreign keys toe.
    - Schrijf de data weg naar de juiste tabellen via bulk-insert of andere batch methoden.

Algemeen:
Voor alle bulk-insert taken moet je zorgen voor een efficiënte schrijfmethode naar SQL Server, bijvoorbeeld:

- to_sql() in combinatie met een SQLAlchemy engine.
- Bulk-insert via pyodbc of tools zoals bcp.
- Gebruik maken van BULK INSERT in SQL Server voor het snel inladen van grote datasets.